# CLIP + LoRA Experiments
## Parameter-Efficient Fine-Tuning

This notebook demonstrates training and evaluating CLIP with LoRA adapters.

## Setup

In [ ]:
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from models.clip_lora import CLIPLoRA
from utils.config import CLIPLoRAConfig
from utils.helpers import seed_everything
from evaluation.metrics import MetricsTracker
from training.trainer_lora import CLIPLoRATrainer

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Configuration

In [ ]:
# Set random seed
seed_everything(42)

# Load configuration
config = CLIPLoRAConfig(
    model_id="openai/clip-vit-base-patch32",
    batch_size=32,
    num_epochs=3,
    learning_rate=5e-5,
    lora_r=16,
    lora_alpha=32,
    seed=42
)

print(f"Device: {config.device}")
print(f"Model: {config.model_id}")
print(f"LoRA Rank: {config.lora_r}")

## Initialize Model

In [ ]:
# Initialize CLIP + LoRA
model = CLIPLoRA(config)

# Print trainable parameters
param_counts = model.count_parameters()
print(f"\nTotal Parameters: {param_counts['total']:,}")
print(f"Trainable Parameters: {param_counts['trainable']:,}")
print(f"Trainable Ratio: {param_counts['trainable']/param_counts['total']*100:.2f}%")

# Initialize metrics tracker
metrics = MetricsTracker("CLIP_LORA_NOTEBOOK", config.results_dir)
metrics.track_parameters(model)

print("\n✓ Model initialized successfully")

## Training

In [ ]:
# Initialize trainer
trainer = CLIPLoRATrainer(model, config, metrics)

# Start training
print("Starting training...\n")
trainer.train()

print("\n✓ Training complete!")

## Evaluation

In [ ]:
# TODO: Add evaluation code for fine-tuned model
# Compare performance before/after fine-tuning

print("Evaluation code to be implemented...")

## Analysis: LoRA Adapter Weights

In [ ]:
# Visualize LoRA adapter weight distributions
lora_weights = []

for name, param in model.model.named_parameters():
    if 'lora' in name and param.requires_grad:
        lora_weights.append(param.detach().cpu().numpy().flatten())

if lora_weights:
    fig, axes = plt.subplots(1, len(lora_weights[:4]), figsize=(16, 4))
    
    for idx, (ax, weights) in enumerate(zip(axes, lora_weights[:4])):
        ax.hist(weights, bins=50, alpha=0.7)
        ax.set_title(f'LoRA Layer {idx+1}')
        ax.set_xlabel('Weight Value')
        ax.set_ylabel('Frequency')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../plots/lora_weight_distributions.png', dpi=300)
    plt.show()
else:
    print("No LoRA weights found")

## Save Results

In [ ]:
# Save metrics
metrics.save_metrics(run_id="notebook_experiment")
metrics.print_summary()

print("\n✓ Results saved successfully!")